### Clean covid data 

1. get the covid data from api and read it in using pandas

2. subset the dataset to keep rows with the last available date (as the data is cumulative)

3. clean state names using an apply function and select five columns that will be used for our analysis ('state','positive','hospitalizedCumulative','death','totalTestResults')

4. store the data frame as covid_cumulative

In [1]:
import json
import requests
import pandas as pd
import numpy as np
r = requests.get('https://api.covidtracking.com/v1/states/daily.json')
read_states = r.json()
states_covid_daily = pd.DataFrame(read_states)
states_covid_daily=states_covid_daily[states_covid_daily['date']==20210307]

def clean_state_name(state_acroname):
        state= {
        'AL': 'Alabama',
        'AK': 'Alaska',
        'AS': 'American Samoa',
        'AZ': 'Arizona',
        'AR': 'Arkansas',
        'CA': 'California',
        'CO': 'Colorado',
        'CT': 'Connecticut',
        'DE': 'Delaware',
        'DC': 'District of Columbia',
        'FL': 'Florida',
        'GA': 'Georgia',
        'GU': 'Guam',
        'HI': 'Hawaii',
        'ID': 'Idaho',
        'IL': 'Illinois',
        'IN': 'Indiana',
        'IA': 'Iowa',
        'KS': 'Kansas',
        'KY': 'Kentucky',
        'LA': 'Louisiana',
        'ME': 'Maine',
        'MD': 'Maryland',
        'MA': 'Massachusetts',
        'MI': 'Michigan',
        'MN': 'Minnesota',
        'MS': 'Mississippi',
        'MO': 'Missouri',
        'MT': 'Montana',
        'NE': 'Nebraska',
        'NV': 'Nevada',
        'NH': 'New Hampshire',
        'NJ': 'New Jersey',
        'NM': 'New Mexico',
        'NY': 'New York',
        'NC': 'North Carolina',
        'ND': 'North Dakota',
        'MP':'Northern Mariana Islands',
        'OH': 'Ohio',
        'OK': 'Oklahoma',
        'OR': 'Oregon',
        'PA': 'Pennsylvania',
        'PR': 'Puerto Rico',
        'RI': 'Rhode Island',
        'SC': 'South Carolina',
        'SD': 'South Dakota',
        'TN': 'Tennessee',
        'TX': 'Texas',
        'UT': 'Utah',
        'VT': 'Vermont',
        'VI': 'Virgin Islands',
        'VA': 'Virginia',
        'WA': 'Washington',
        'WV': 'West Virginia',
        'WI': 'Wisconsin',
        'WY': 'Wyoming'}
        if state_acroname in state:
            return state[state_acroname]
        else:
            return state_acroname
        
states_covid_daily['state']=states_covid_daily['state'].apply(clean_state_name)
covid_cumulative=states_covid_daily[['state','positive','hospitalizedCumulative','death','totalTestResults']]


### Clean population data

1. read in the population file(in excel format), adjust column names, and drop NaNs

2. clean values in state column using an apply function

In [2]:
df_population=pd.read_excel("population.xlsx",header=[1],skiprows=[0,1,-1])
df_population=df_population.rename(columns = {'Unnamed: 0':'state',2019:'2019population'})
df_population=df_population[['state', '2019population']].dropna()
def replacing(state):
    state=state.replace('.','')
    return state
df_population['state']=df_population['state'].apply(replacing)

df_population.head()

,state,2019population
0,United States,328239523.0
1,Northeast,55982803.0
2,Midwest,68329004.0
3,South,125580448.0
4,West,78347268.0


### Merging and ratio calculation

1. merge the two datasets together on 'state' column

2. calculate three ratios respectively 

### We will calculate four ratios using the equations listed below
1. Death rate = 'death' / 'positive'
2. Hospitalized rate = 'hospitalizedCumulative' / 'positive'
3. Infection rate = 'positive' / 'population' (from population dataset)
4. Test rate = 'totalTestResults' () / 'population' (from population dataset)

In [3]:
Covid_and_population=pd.merge(df_population,covid_cumulative,how='inner',on='state')
Covid_and_population['infection_rate']=Covid_and_population['positive']/Covid_and_population['2019population']
Covid_and_population['death_rate']=Covid_and_population['death']/Covid_and_population['positive']
Covid_and_population['test_rate']=Covid_and_population['totalTestResults']/Covid_and_population['2019population']
Covid_and_population['hospitalized_rate']=Covid_and_population['hospitalizedCumulative']/Covid_and_population['positive']
# Covid_and_population['hospitalized_rate'].fillna(value=np.(Covid_and_population['hospitalized_rate']), inplace=True)
Covid_and_population.head()

,state,2019population,positive,hospitalizedCumulative,death,totalTestResults,infection_rate,death_rate,test_rate,hospitalized_rate
0,Alabama,4903185.0,499819.0,45976.0,10148.0,2323788.0,0.101938,0.020303,0.473934,0.091985
1,Alaska,731545.0,56886.0,1293.0,305.0,1731628.0,0.077761,0.005362,2.367083,0.022730
2,Arizona,7278717.0,826454.0,57907.0,16328.0,7908105.0,0.113544,0.019757,1.086470,0.070067
3,Arkansas,3017804.0,324818.0,14926.0,5319.0,2736442.0,0.107634,0.016375,0.906766,0.045952
4,California,39512223.0,3501394.0,NaN,54124.0,49646014.0,0.088615,0.015458,1.256472,NaN


### Calculate the final score

1. using qcut to assgin each state into 4 quartiles respectively

2. assign each quartile with different scores using an apply function and store the final_score in a new column

3. export the data frame as a csv file for further analysis and visualization

In [4]:
labels = pd.Categorical(['lowest25%','50%-75%','''25%-50%''','top25%'],ordered=True)

Covid_and_population['infection_rank'] = pd.qcut(Covid_and_population['infection_rate'], 4, labels = labels)
Covid_and_population['death_rank'] = pd.qcut(Covid_and_population['death_rate'], 4, labels=labels)
Covid_and_population['test_rank'] = pd.qcut(Covid_and_population['test_rate'], 4,labels=labels)

def final_score(row):
    score = int()
    if row[-1]=='top25%':
        score += 100
    elif row[-1]=='25%-50%':
        score += 75
    elif row[-1]=='50%-75%':
        score += 50
    elif row[-1]=='lowest25%':
        score += 25
    if row[-2]=='top25%':
        score += 25
    elif row[-2]=='25%-50%':
        score += 50
    elif row[-2]=='50%-75%':
        score += 75
    elif row[-2]=='lowest25%':
        score += 100
    if  row[-3]=='top25%':
        score += 25
    elif row[-3]=='25%-50%':
        score += 50
    elif row[-3]=='50%-75%':
        score += 75
    elif row[-3]=='lowest25%':
        score += 100
    return score

Covid_and_population['final_score'] = Covid_and_population.apply(final_score, axis=1)
Covid_and_population['final_score']
Covid_and_population=Covid_and_population.sort_values(['final_score','infection_rate'],ascending=False)
Covid_and_population=Covid_and_population.reset_index().drop('index',axis=1)
Covid_and_population.index+=1
Covid_and_population.to_csv('output.csv')